<a href="https://colab.research.google.com/github/tongni1975/100-Days-Of-ML-Code/blob/master/1_1_Spacy_an_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spacy basic NLP tasks

In this tutorial, we introduce basic NLP tasks that are associated with Spacy. Spacy is selected as it is an industrial-grade NLP library that is known to be fastest and most accurate in the industry and research. 

The reference for this tutorial is from https://spacy.io/usage/. Some of the material is obtained from the spacy notebooks therein.

The Spacy library is described as 4 sections - usage, models, api and universe. The set-up instructions are in the usage, while the model section looks into the models for tagging, parsing and entity recognition. There are no inherent models to do training for sentiment etc, which still need to call on libraries like sci-kit learn. 

## Architecture
The Spacy library is useful as a building block to build NLP libraries - a valuable skillset. To do so, a basic understanding of its architecture is necessary. A pictorial of it below. Spacy comes with models for its 'Language' models - which are used to build the 'Vocab' that is particular to your use case. Further, the 'Doc' as represented below are your own training or test documents. The Doc is broken into tokens (words or groups of words), span (a subset of documents) or lexeme (refers to the root words of words).

In [4]:
from os import chdir, getcwd
fpath = getcwd()
print (fpath)
from IPython.display import Image
Image(filename=fpath + '\\images\\architecture.PNG', width=550, height=400)

/content


FileNotFoundError: ignored

In [2]:
# this is to make sure we get no unicode based errors
from __future__ import unicode_literals
import spacy
import numpy as np
print (spacy.__version__)

2.1.8


In [3]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


## Containers in Spacy
There are 4 key containers in spacy - 
1. Tokens - they are simplest and contain many useful different properties.
2. doc - this is a sequence of tokens
3. span - this is a slice from a doc object 
4. lexme - an entry in the vocabulary

First we load the spacy Language model (English). In order to process sentences, we need to use the model 'nlp', which does parsing, POS, NER on the sentence on each of the tokens. We explore the properites of these tokens.

## Tokens

A token is a single chopped up element of the sentence, which could be a word or a group of words to analyse. The task of chopping the sentence up is called "tokenisation".

Example: The following sentence can be tokenised by splitting up the sentence into individual words.

	"Cytora is going to PyCon!"
	["Cytora","is","going","to","PyCon!"]

In [5]:
# u need to run for the spacy models
# python -m spacy download en_core_web_sm or md
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'Hello world! Natural Language Processing 101 is f8unn')

# Get first token of the processed document
token = doc[0]
print(token)

# Print sentences (one sentence per line)
for sent in doc.sents:
    print(sent)

Hello
Hello world!
Natural Language Processing 101 is f8unn


In [7]:
print(token.orth)  # token uinque id. These are stored as hashes ID/
print(token.orth_)

15777305708150031551
Hello


### Token tags

A POS tag is a context sensitive description of what a word means in the context of the whole sentence.
More information about the kinds of speech tags which are used in Spacy can be [found here](https://spacy.io/api/annotation#section-pos-tagging).

Examples:

1. CARDINAL, Cardinal Number - 1,2,3
2. PROPN, Proper Noun, Singular - "Matic", "Andraz", "Cardiff"
3. INTJ, Interjection - "Uhhhhhhhhhhh"

The following are some tags associated with tokens:
- Text: The original word text.
- Lemma: The base form of the word.
- POS: The simple part-of-speech tag.
- Tag: The detailed part-of-speech tag.
- Dep: Syntactic dependency, i.e. the relation between tokens.
- Shape: The word shape – capitalization, punctuation, digits.
- is alpha: Is the token an alpha character?
- is stop: Is the token part of a stop list, i.e. the most common words of the language?

For more properties, refer to https://spacy.io/api/token

In [8]:
# For each token, print corresponding part of speech tag
for token in doc:
    print('{} - {}'.format(token, token.pos_), token.has_vector, token.vector_norm, token.is_oov)
    

Hello - INTJ True 24.419973 True
world - NOUN True 22.475723 True
! - PUNCT True 25.938637 True
Natural - PROPN True 23.645964 True
Language - PROPN True 24.362488 True
Processing - PROPN True 25.772451 True
101 - NUM True 24.622292 True
is - VERB True 26.335238 True
f8unn - PUNCT True 20.704897 True


In [9]:
from spacy.parts_of_speech import ADV

def is_adverb(token):
    return token.pos == spacy.parts_of_speech.ADV

NNS = nlp.vocab.strings['NNS']
NNPS = nlp.vocab.strings['NNPS']
def is_plural_noun(token):
    return token.tag == NNS or token.tag == NNPS
    
doc = nlp(u"Apples and orange are similar.")
for token in doc:
    print(token, token.pos_, token.tag_, is_adverb(token), is_plural_noun(token), token.sentiment)
# for more properties, refer to https://spacy.io/api/token

Apples NOUN NNS False True 0.0
and CCONJ CC False False 0.0
orange NOUN NN False False 0.0
are VERB VBP False False 0.0
similar ADJ JJ False False 0.0
. PUNCT . False False 0.0


### Token unigram probability

In [10]:
# For every token in doc, print log-probability of the word, estimated from counts from the reference model Need to use a larger model to observe
for token in doc:
    print(token, ',', np.exp(token.prob))                                     #how often does the word occur

Apples , 2.061153622438558e-09
and , 2.061153622438558e-09
orange , 2.061153622438558e-09
are , 2.061153622438558e-09
similar , 2.061153622438558e-09
. , 2.061153622438558e-09


## Doc object

From the doc, it is possible to obtain named entities, which is any real world object such as a person, location, organisation or product with a proper name. The doc object also has noun chunks. Noun chunks are the phrases based upon nouns recovered from tokenized text using the speech tags.

Example:

	1. Barack Obama
	2. Edinburgh
	3. Ferrari Enzo

In [11]:
# Print all named entities with named entity types
doc_2 = nlp(u"Barack Obama lived in Washington and liked to eat Burger King.")  # ths create a doc object which has ents
for ent in doc_2.ents:
    print('{} - {}'.format(ent, ent.label_))
    
doc_3 = nlp(u"The boy loves to play football.") 
print([chunk for chunk in doc_3.noun_chunks])

Barack Obama - PERSON
Washington - GPE
Burger King - ORG
[The boy, football]


#### Export to numpy arrays

In [0]:
from spacy.attrs import ORTH, LIKE_URL, IS_OOV

attr_ids = [ORTH, LIKE_URL, IS_OOV]
doc_array = doc_2.to_array(attr_ids)
assert doc_array.shape == (len(doc_2), len(attr_ids))
assert doc_2[0].orth == doc_array[0, 0]
assert doc_2[1].orth == doc_array[1, 0]
assert doc_2[0].like_url == doc_array[0, 1]
assert list(doc_array[:, 1]) == [t.like_url for t in doc_2]

## Vocabulary
A vocabulary is a collection of lexeme - which are the root of words. It is also possible to extract all words from the built-in vocabulary, but memory will overflow below. The vocabulary is useful in builing training data sets. Here we create a new vocabulary from some words. 

In [14]:
# Vocabulary from spacy model
apple_id = nlp.vocab.strings['apple']   # nlp is the language model
oov = nlp.vocab.strings['dskfodkfos']

print (apple)  # the words are identified as hashes in the vocabulary
print (oov)

apple_str = nlp.vocab.strings[apple_id]
print (apple_str)
assert oov not in nlp.vocab

# Create new vocabulary of two words
from spacy.vocab import Vocab
new_vocab = Vocab(strings=[u'hello', u'world'])   
for i in new_vocab:
    print(i.text, i.orth)

NameError: ignored

In [15]:
# printing out some of the stop words in the model vocabulary
from spacy.lang.en.stop_words import STOP_WORDS
print(STOP_WORDS) # <- set of Spacy's default stop words


{'serious', 'between', 'he', 'us', 'used', 'whereas', 'hers', 'only', 'until', 'your', 'hundred', '‘s', 'hereafter', 'thereafter', 'so', 'can', 'name', 'unless', 'during', 'side', 'somewhere', 'therefore', 'therein', 'third', 'beforehand', 'six', 'why', 'often', 'various', 'eleven', 'alone', 'others', 'on', 'across', 'about', 'even', 'part', 'am', 'together', 'be', 'amount', 'less', 'these', 'throughout', 'than', 'well', 'will', 'you', 'all', 'say', 'none', 'thus', 'whenever', 'whether', 'yourself', 'myself', 'put', 'former', 'seemed', '’d', 'seems', 'almost', 'anyway', 'whole', 'anywhere', 'towards', 'was', 'ever', 'were', 'already', 'though', 'whereby', 'would', 'also', 'its', 'besides', 'had', "'m", 'n’t', 'it', 'whatever', 'just', 'who', 'becoming', 'whence', 'against', 'because', 'quite', 'neither', 'been', 'might', 'mine', 'front', 'further', 'then', 'really', 'call', 'last', 'fifteen', 'never', 'same', 'whoever', 'bottom', 'if', 'no', 'rather', 'that', 'under', 'onto', 'from', '

### n-grams using NLTK

Spacy however does not have a built-in library to do n-gram tokenisation. This is done much easier using nltk.

In [0]:
import nltk

In [17]:
# in the below we obtain pairs of JJ-NN
def ngramise(sequence):
    '''
    Iterate over bigrams and 1,2-skip-grams.
    '''
    for bigram in nltk.ngrams(sequence, 2):
        yield bigram
    for trigram in nltk.ngrams(sequence, 3):
        yield trigram[0], trigram[2]

def jjnn_pairs(phrase):
    '''
    Iterate over pairs of JJ-NN.                     #jj refers to adjectives and NN refers to nouns
    '''
    tagged = nltk.pos_tag(nltk.word_tokenize(phrase))
    for ngram in ngramise(tagged):
        tokens, tags = zip(*ngram)
        if tags == ('JJ', 'NN'):
            yield tokens

       
for gram in jjnn_pairs("This is a delicious apple"):
    print (gram)

LookupError: ignored

## Word embedding / Similarity

A word embedding is a representation of a word, and by extension a whole language corpus, in a vector or other form of numerical mapping. This allows words to be treated numerically with word similarity represented as spatial difference in the dimensions of the word embedding mapping. The word vectors can be used as training features and will be described in greater detail in Day 2 (and also previously in Text Analytics).

Example:
	
With word embeddings we can understand that vector operations describe word similarity. This means that we can see vector proofs of statements such as:

	king-queen==man-woman

In [19]:
doc = nlp(u"Apples and orange are similar. Hippos and lions are not.")
for token in doc:
    print(token.text, token.has_vector, token.vector_norm)  
    print(token.vector)

Apples True 24.373877
[ 6.48012     0.4459746   0.8411504  -2.4217246  -0.81870496  0.39663044
  4.8589735   4.484127   -3.0453458   2.6891198   1.4677203  -0.26530552
 -3.7863774  -1.2364017  -3.9447026  -0.33001122  0.88959736  0.97850204
 -1.7332234   0.3293991  -1.8573451  -0.00700772 -1.0143324   0.89394915
  0.02612577 -0.00712687  1.8551004  -3.123006    6.8388224   0.781844
  3.97846    -1.882559   -0.73632044  0.08226132  1.7286662   2.0666804
 -3.7042937   5.114806   -0.32460582  2.112061   -2.709859   -0.29375124
  3.9201634  -2.1118784   5.2576485  -4.1561074  -1.9643553  -1.3030659
  0.5393265  -2.40053    -5.5046544  -0.23191261 -1.4842712  -0.1768856
  1.2075411   1.0816665  -0.20792592 -1.4299974  -4.481552   -2.4571939
 -0.9589004  -0.24006546  4.030328   -1.1740503   0.5638145   6.994032
 -1.0593221   1.3389046   0.22301637  1.8767828  -2.571108    0.8126358
  1.5665478   1.6212194   0.3533219  -1.0702294   1.2023344  -0.03223666
  1.6001799  -2.273003   -3.6745358  -

In [20]:
# add stop words
nlp.Defaults.stop_words -= {"are", ".", "and"}

# Finding the similarity between words    ##Cosine similarity

for token1 in doc:
    for token2 in doc:
        if not token1.is_stop and not token2.is_stop and token1 != token2:
            print(token1.text, token2.text, token1.similarity(token2))

/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instea

Apples orange 0.33165458
Apples similar 0.096144706
Apples . 0.021095317
Apples Hippos 0.4646771
Apples lions 0.5326731
Apples . -0.058706053
orange Apples 0.33165458
orange similar 0.24862732
orange . -0.039233293
orange Hippos 0.35312775
orange lions 0.5714338
orange . -0.06659413
similar Apples 0.096144706
similar orange 0.24862732
similar . -0.0061813476
similar Hippos 0.20505269
similar lions -0.0695914
similar . 0.118038565
. Apples 0.021095317
. orange -0.039233293
. similar -0.0061813476
. Hippos 0.10610138
. lions 0.07583006
. . 1.0
Hippos Apples 0.4646771
Hippos orange 0.35312775
Hippos similar 0.20505269
Hippos . 0.10610138
Hippos lions 0.2377463
Hippos . 0.083698876
lions Apples 0.5326731
lions orange 0.5714338
lions similar -0.0695914
lions . 0.07583006
lions Hippos 0.2377463
lions . 0.031110972
. Apples -0.058706053
. orange -0.06659413
. similar 0.118038565
. . 1.0
. Hippos 0.083698876
. lions 0.031110972


/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instea

## Syntactic dependencies

#### What are syntactic dependencies?

We have the speech tags and we have all of the tokens in a sentence, but how do we relate the two to uncover the syntax in a sentence? Syntactic dependencies describe how each type of word relates to each other in a sentence, this is important in NLP in order to extract structure and understand grammar in plain text.

Example:

<img src="images/syntax-dependencies-oliver.png" align="left" width=500>

In [21]:
doc = nlp(u'The three monkeys ate a bunch of delicious bananas.')
print(doc)

The three monkeys ate a bunch of delicious bananas.


In [0]:
from spacy import displacy
displacy.serve(doc, style="dep")


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



#### Using WordNet to change sentences expression

This is similar to Valentino, in using WordNet to change expressions to have a higher valence.

In [0]:
#from spacy_wordnet.wornet_annotator import WordnetAnnotator 
import spacy_wordnet
from spacy_wordnet.wordnet_annotator import WordnetAnnotator

# nltk.download("wordnet")

In [0]:
if "WordnetAnnotator" not in nlp.pipe_names:
    nlp.add_pipe(WordnetAnnotator(nlp.lang), after="tagger")
print("after", nlp.pipe_names)


after ['tagger', 'WordnetAnnotator', 'parser', 'ner']


In [0]:
token = nlp("beautiful")[0]  # try with other words like 'bank' etc
token_ = token._.wordnet.synsets()
token_

[Synset('beautiful.a.01'), Synset('beautiful.s.02')]

In [0]:
token._.wordnet.lemmas()

[Lemma('beautiful.a.01.beautiful'), Lemma('beautiful.s.02.beautiful')]

In [0]:
token._.wordnet.wordnet_domains()

['philosophy',
 'quality',
 'mythology',
 'painting',
 'plants',
 'art',
 'body_care',
 'person',
 'fashion',
 'graphic_arts',
 'drawing',
 'archaeology',
 'meteorology']

In [0]:
domains = ["painting","art","graphic_arts"]
sentence = nlp("This beautiful picture looks awesome.")

enriched_sent = []
 
for token in sentence:
    # get synsets within the desired domains
    synsets = token._.wordnet.wordnet_synsets_for_domain(domains)
    synsets = token._.wordnet.synsets()
    if synsets and token.pos == spacy.parts_of_speech.ADJ:
       lemmas_for_synset = []
 
       for s in synsets:
           # get synset variants and add to the enriched sentence
           lemmas_for_synset.extend(s.lemma_names())
           enriched_sent.append("({})".format("|".join(set(lemmas_for_synset))))
    else:
        enriched_sent.append(token.text)
 
print(" ".join(enriched_sent))

This (beautiful) (beautiful) picture looks (amazing|awe-inspiring|awesome|awful|awing) .


In [0]:
### Placeholder only for reference

### Multi-threaded generator
#texts = [u'One document.', u'...', u'Lots of documents']
# .pipe streams input, and produces streaming output
#iter_texts = (texts[i % 3] for i in range(1000))
#for i, doc in enumerate(nlp.pipe(iter_texts, batch_size=50, n_threads=4)):
#    assert doc.is_parsed
#    if i == 100:
#        break

#Efficient binary serialization
#from spacy.tokens.doc import Doc
#byte_string = doc.to_bytes()
#open('moby_dick.bin', 'wb').write(byte_string)
#for byte_string in Doc.read_bytes(open('moby_dick.bin', 'rb')):
#   doc = Doc(nlp.vocab)
#   doc.from_bytes(byte_string)